In [1]:
###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############

with open('example.txt', 'r') as file:
    maze = [list(line.strip()) for line in file.readlines()]

    coordinates: dict[tuple[int, int], str] = {}
    for y, row in enumerate(maze):
        for x, char in enumerate(row):
            coordinates[(x, y)] = char


In [2]:
def get_start(coords):
    for coord, char in coords.items():
        if char == 'S':
            return coord
        
def get_end(coords):
    for coord, char in coords.items():
        if char == 'E':
            return coord
start = get_start(coordinates)
end = get_end(coordinates)
start_dir = (1, 0)

print(start, end, start_dir)

(1, 13) (13, 1) (1, 0)


In [3]:
class DirectedEdge:
    def __init__(self, start: tuple, end: tuple, weight: int):
        self.start = start
        self.end = end
        self.weight = weight

    def coordinates(self) -> tuple[tuple, tuple]:
        return self.start, self.end

    def __repr__(self):
        return f'{self.start} -[{self.weight}]-> {self.end}'

    def __eq__(self, other):
        return self.start == other.start and self.end == other.end

    def __hash__(self):
        return hash((self.start, self.end))
    
class EdgeWeightedDigraph:
    adj: dict[tuple[int, int], list[DirectedEdge]] = {}

    def __init__(self):
        self.adj = {}

    def add_edge(self, edge: DirectedEdge):
        if edge.start not in self.adj:
            self.adj[edge.start] = []
        if edge.end not in self.adj:
            self.adj[edge.end] = []
        self.adj[edge.start].append(edge)

    def neighbours(self, vertex: tuple) -> list[DirectedEdge]:
        return self.adj.get(vertex, [])

    def vertices(self):
        return self.adj.keys()

    def edges(self) -> list[DirectedEdge]:
        edges = []
        for vertex in self.adj:
            for edge in self.adj[vertex]:
                edges.append(edge)
        return edges
    
    def find_edge_with_start(self, start: tuple) -> list[DirectedEdge]:
        edges = []
        for edge in self.edges():
            if edge.start == start:
                edges.append(edge)
        return edges
    
    def find_edge_with_end(self, end: tuple) -> list[DirectedEdge]:
        edges = []
        for edge in self.edges():
            if edge.end == end:
                edges.append(edge)
        return edges

    def __repr__(self):
        return str(self.adj)

In [4]:
def min_max_coordinates(coordinates):
    min_x = min_y = float('inf')
    max_x = max_y = float('-inf')

    for x, y in coordinates:
        min_x = min(min_x, x)
        min_y = min(min_y, y)
        max_x = max(max_x, x)
        max_y = max(max_y, y)

    return (min_x, min_y), (max_x, max_y)

def add(tuple, tuple2):
    return tuple[0] + tuple2[0], tuple[1] + tuple2[1]

def is_walkable(coords, x, y):
    return (x, y) in coords and (coords[(x, y)] == '.' or coords[(x, y)] == 'E' or coords[(x, y)] == 'S')


def create_edges_from_coordinates(coordinates) -> list[DirectedEdge]:
    edges = []
    directions = [(1, 0), (0, 1), (-1, 0), (0, -1)]  # right, down, left, up
    min_coord, max_coord = min_max_coordinates(coordinates)

    for x in range(min_coord[0], max_coord[0] + 1):
        for y in range(min_coord[1], max_coord[1] + 1):
            if (x, y) not in coordinates:
                continue
            if coordinates[(x, y)] == '#':
                continue

            neighbours_vertical = []
            neighbours_horizontal = []
            for dx, dy in directions:
                nx, ny = x + dx, y + dy # right, down, left, up
                if is_walkable(coordinates, nx, ny):
                    if dx == 0:
                        neighbours_vertical.append((nx, ny))
                    else:
                        neighbours_horizontal.append((nx, ny))
            if len(neighbours_horizontal) == 2 and len(neighbours_vertical) == 2:
                for i in range(len(neighbours_horizontal)):
                    edges.append(DirectedEdge(neighbours_horizontal[i], (x,y), 1))
                    for j in range(len(neighbours_vertical)):
                        edges.append(DirectedEdge(neighbours_horizontal[i], neighbours_vertical[j], 1002))
                        edges.append(DirectedEdge(neighbours_vertical[j], neighbours_horizontal[i], 1002))
                for j in range(len(neighbours_vertical)):
                    edges.append(DirectedEdge(neighbours_vertical[j], (x,y), 1))
                edges.append(DirectedEdge(neighbours_horizontal[0], neighbours_horizontal[1], 2))
                edges.append(DirectedEdge(neighbours_horizontal[1], neighbours_horizontal[0], 2))
                edges.append(DirectedEdge(neighbours_vertical[1], neighbours_vertical[0], 2))
                edges.append(DirectedEdge(neighbours_vertical[0], neighbours_vertical[1], 2))

            elif len(neighbours_horizontal) == 2 and len(neighbours_vertical) == 1:
                for i in range(len(neighbours_horizontal)):
                    edges.append(DirectedEdge(neighbours_horizontal[i], neighbours_vertical[0], 1002))
                    edges.append(DirectedEdge(neighbours_vertical[0], neighbours_horizontal[i], 1002))
                    edges.append(DirectedEdge(neighbours_horizontal[i], (x,y), 1))
                    edges.append(DirectedEdge(neighbours_horizontal[0], neighbours_horizontal[1], 2))
                    edges.append(DirectedEdge(neighbours_horizontal[1], neighbours_horizontal[0], 2))
                edges.append(DirectedEdge(neighbours_vertical[0], (x,y), 1))

            elif len(neighbours_horizontal) == 1 and len(neighbours_vertical) == 2:
                for i in range(len(neighbours_vertical)):
                    edges.append(DirectedEdge(neighbours_horizontal[0], neighbours_vertical[i], 1002))
                    edges.append(DirectedEdge(neighbours_vertical[i], neighbours_horizontal[0], 1002))
                    edges.append(DirectedEdge(neighbours_vertical[i], (x,y), 1))
                    edges.append(DirectedEdge(neighbours_vertical[0], neighbours_vertical[1], 2))
                    edges.append(DirectedEdge(neighbours_vertical[1], neighbours_vertical[0], 2))
                edges.append(DirectedEdge(neighbours_horizontal[0], (x,y), 1))

            elif len(neighbours_horizontal) == 1 and len(neighbours_vertical) == 1:
                edges.append(DirectedEdge(neighbours_horizontal[0], neighbours_vertical[0], 1002))
                edges.append(DirectedEdge(neighbours_vertical[0], neighbours_horizontal[0], 1002))
                edges.append(DirectedEdge(neighbours_horizontal[0], (x,y), 1))
                edges.append(DirectedEdge(neighbours_vertical[0], (x,y), 1))

            elif len(neighbours_horizontal) == 2 and len(neighbours_vertical) == 0:
                flags = 0
                for i in range(len(neighbours_horizontal)):
                    if is_walkable(coordinates, *add(neighbours_horizontal[i], (0, 1))) or is_walkable(coordinates, *add(neighbours_horizontal[i], (0, -1))):
                    # if (add(neighbours_horizontal[i], (0, 1)) in coordinates and coordinates[add(neighbours_horizontal[i],(0, 1))] == '.') or (add(neighbours_horizontal[i],(0, -1)) in coordinates and coordinates[add(neighbours_horizontal[i], (0, -1))] == '.'):
                        flags += 1
                    else:
                        edges.append(DirectedEdge((x, y), neighbours_horizontal[i], 1))
                if flags == 0:
                    edges.append(DirectedEdge(neighbours_horizontal[i], add(neighbours_horizontal[i],(0, 1)), 1))

                    


            elif len(neighbours_horizontal) == 0 and len(neighbours_vertical) == 2:
                flags = 0
                for i in range(len(neighbours_vertical)):
                    if is_walkable(coordinates, *add(neighbours_vertical[i], (1, 0))) or is_walkable(coordinates, *add(neighbours_vertical[i], (-1, 0))):
                    # if (add(neighbours_vertical[i], (1, 0)) in coordinates and coordinates[add(neighbours_vertical[i], (1, 0))] == '.') or (add(neighbours_vertical[i], (-1, 0)) in coordinates and coordinates[add(neighbours_vertical[i], (-1, 0))] == '.'):
                        flags += 1
                    else:
                        edges.append(DirectedEdge((x, y), neighbours_vertical[i], 1))
                        pass
                if flags == 0:
                    edges.append(DirectedEdge(neighbours_vertical[i], (x,y), 1))
                    

            elif len(neighbours_horizontal) == 1 and len(neighbours_vertical) == 0:
                if (add(neighbours_horizontal[0], (0, 1)) in coordinates and coordinates[add(neighbours_horizontal[0], (0, 1))] == '.') or (add(neighbours_horizontal[0], (0, -1)) in coordinates and coordinates[add(neighbours_horizontal[0], (0, -1))] == '.'):
                    pass
                else:
                    edges.append(DirectedEdge( neighbours_horizontal[0], (x,y), 1))
                # edges.append(DirectedEdge((x,y), neighbours_horizontal[0], 1))

            elif len(neighbours_horizontal) == 0 and len(neighbours_vertical) == 1:
                if (add(neighbours_vertical[0], (1, 0)) in coordinates and coordinates[add(neighbours_vertical[0], (1, 0))] == '.') or (add(neighbours_vertical[0], (-1, 0)) in coordinates and coordinates[add(neighbours_vertical[0], (-1, 0))] == '.'):
                    pass
                else:
                    edges.append(DirectedEdge(neighbours_vertical[0], (x,y),  1))

            if coordinates[(x, y)] == 'S':
                right = add((x, y), (1, 0))
                down = add((x, y), (0, 1))
                left = add((x, y), (-1, 0))
                up = add((x, y), (0, -1))
                if is_walkable(coordinates, *right):
                    edges.append(DirectedEdge((x, y), right, 1))
                if is_walkable(coordinates, *down):
                    edges.append(DirectedEdge((x, y), down, 1001))
                if is_walkable(coordinates, *left):
                    edges.append(DirectedEdge((x, y), left, 2001))
                if is_walkable(coordinates, *up):
                    edges.append(DirectedEdge((x, y), up, 1001))

    return list(set(edges))



# Create the graph and add edges
def create_graph(coordinates) -> EdgeWeightedDigraph:
    # Create edges from coordinates
    edges = create_edges_from_coordinates(coordinates)

    graph = EdgeWeightedDigraph()
    for edge in edges:
        graph.add_edge(edge)
    return graph

graph = create_graph(coordinates)

print(graph)

print(graph.find_edge_with_start((7,3)))
print(graph.find_edge_with_start((7,2)))
print(graph.find_edge_with_start((7,1)))
print(graph.find_edge_with_start((1,3)))
print(graph.find_edge_with_start((1,13)))

{(10, 3): [(10, 3) -[1002]-> (11, 4), (10, 3) -[1002]-> (9, 2), (10, 3) -[1]-> (9, 3), (10, 3) -[1]-> (11, 3)], (11, 4): [(11, 4) -[1]-> (11, 3), (11, 4) -[1002]-> (10, 3), (11, 4) -[1]-> (11, 5), (11, 4) -[1002]-> (10, 5)], (3, 6): [(3, 6) -[2]-> (3, 8), (3, 6) -[1002]-> (2, 7), (3, 6) -[1002]-> (4, 7), (3, 6) -[1]-> (3, 7), (3, 6) -[1]-> (3, 5)], (3, 8): [(3, 8) -[2]-> (3, 10), (3, 8) -[1]-> (3, 9), (3, 8) -[1002]-> (2, 9), (3, 8) -[1002]-> (4, 7), (3, 8) -[1]-> (3, 7), (3, 8) -[1002]-> (2, 7), (3, 8) -[2]-> (3, 6)], (7, 12): [(7, 12) -[1]-> (7, 13), (7, 12) -[1002]-> (8, 11), (7, 12) -[1002]-> (6, 13), (7, 12) -[1]-> (7, 11), (7, 12) -[1002]-> (8, 13)], (7, 13): [], (2, 13): [(2, 13) -[1]-> (1, 13), (2, 13) -[1]-> (3, 13), (2, 13) -[1002]-> (1, 12)], (1, 13): [(1, 13) -[1]-> (2, 13), (1, 13) -[1001]-> (1, 12)], (4, 1): [(4, 1) -[2]-> (2, 1), (4, 1) -[1]-> (4, 2), (4, 1) -[1]-> (3, 1), (4, 1) -[1002]-> (3, 2), (4, 1) -[1]-> (5, 1)], (2, 1): [(2, 1) -[1002]-> (1, 2), (2, 1) -[1]-> (1,

In [5]:
graph.neighbours((7,2))

[(7, 2) -[1]-> (7, 3), (7, 2) -[1002]-> (6, 1), (7, 2) -[1]-> (7, 1)]

In [6]:
pq: list[tuple[tuple[int, int], int]] = []
pq.append(((0,0), 9))
pq.append(((0,1), 5))
pq.append(((0,1), 6))

min_key = min(pq, key=lambda x: x[1])
print(min_key)

pq.remove(min_key)
print(pq)

((0, 1), 5)
[((0, 0), 9), ((0, 1), 6)]


In [8]:
class IndexMinPQ:
    pq: list[tuple[tuple[int, int], int]] = []
    def __init__(self):
        pass

    def is_empty(self):
        return len(self.pq) == 0
    
    def insert(self, key: tuple[int, int], distance: int):
        self.pq.append((key, distance))

    def del_min(self) -> tuple[int, int]:
        min_key = min(self.pq, key=lambda x: x[1])
        self.pq.remove(min_key)
        return min_key[0]
    
    def contains(self, key: tuple[int, int]):
        return key in [x[0] for x in self.pq]
    
    def change(self, key: tuple[int, int], distance: int):
        for i, (k, d) in enumerate(self.pq):
            if k == key:
                self.pq[i] = (key, distance)
                break
    


class DijkstraSP:
    edge_to: dict[tuple[int, int], DirectedEdge] = {}
    edges_to: dict[tuple[int, int], list[DirectedEdge]] = {}
    dist_to: dict[tuple[int, int], int] = {}
    
    pq = IndexMinPQ()
    mystart: tuple = None

    def __init__(self, graph: EdgeWeightedDigraph, start: tuple[int, int]):
        self.mystart = start
        for vertex in graph.vertices():
            self.dist_to[vertex] = float('inf')
        self.dist_to[start] = 0

        self.pq.insert(start, 0)
        while not self.pq.is_empty():
            vertex = self.pq.del_min()
            self.relax(graph, vertex)

    def relax(self, graph: EdgeWeightedDigraph, vertex: tuple[int, int]):
        for edge in graph.neighbours(vertex):

            w = edge.end
            if self.dist_to[w] > self.dist_to[vertex] + edge.weight:
                self.dist_to[w] = self.dist_to[vertex] + edge.weight
                self.edge_to[w] = edge
                self.edges_to[w] = [edge]
                if self.pq.contains(w):
                    self.pq.change(w, self.dist_to[w])
                else:
                    self.pq.insert(w, self.dist_to[w])
            if self.dist_to[w] == self.dist_to[vertex] + edge.weight:
                if edge not in self.edges_to[w]:
                    self.edges_to[w].append(edge)

    def has_path_to(self, end: tuple):
        d = self.dist_to.get(end, None)
        if d is None:
            return False
        return d < float('inf')

    def path_to(self, x: tuple) -> list[DirectedEdge] | None:
        if not self.has_path_to(end):
            return None
        path: list[DirectedEdge] = []
        edge = self.edge_to[x]
        while edge is not None:
            path.append(edge)
            edge = self.edge_to.get(edge.start)
        return list(reversed(path))
    
    def roadblock_locations(self) -> dict[tuple[int, int], list[tuple[int, int]]]:
        crossroads = {}
        for k, val in self.edges_to.items():
            if len(val) > 1:
                crossroads[k] = [edge.start for edge in val]
        return crossroads
    
    # def paths_to(self, x: tuple) -> list[DirectedEdge] | None:
    #     if not self.has_path_to(end):
    #         return None
    #     paths: list[DirectedEdge] = []
    #     edges = self.edges_to[x] if x in self.edges_to else []
    #     for edge in edges:
    #         if edge is not None:
    #             paths.append(edge)
    #             paths.extend(self.paths_to(edge.start))
            
    #     return paths
            

    def dist_to_end(self, end: tuple):
        return self.dist_to[end]

    def __repr__(self):
        return str(self.edge_to)
    
dijkstra = DijkstraSP(graph, start)
dijkstra.edge_to[(1, 12)]

(1, 13) -[1001]-> (1, 12)

# Part 2


In [22]:
coordinates: dict[tuple[int, int], str] = {}
with open('input.txt', 'r') as file:
    maze = [list(line.strip()) for line in file.readlines()]

    for y, row in enumerate(maze):
        for x, char in enumerate(row):
            coordinates[(x, y)] = char

def get_start(coords):
    for coord, char in coords.items():
        if char == 'S':
            return coord
        
def get_end(coords):
    for coord, char in coords.items():
        if char == 'E':
            return coord
start = get_start(coordinates)
end = get_end(coordinates)
start_dir = (1, 0)

print(start, end, start_dir)

(1, 139) (139, 1) (1, 0)


In [27]:
import copy

graph = create_graph(coordinates)
dijkstra = DijkstraSP(graph, start)

path = dijkstra.path_to(end)
dist = dijkstra.dist_to_end(end)
print("distance:", dist)

def print_maze(maze, path_coords):
    for y in range(len(maze)):
        for x in range(len(maze[0])):
            if (x, y) in path_coords:
                print('o', end='')
            else:
                print(maze[y][x], end='')
        print()

def get_path_coords(p):
    coords = []
    prev_dir = (1, 0)
    for edge in p:
        coords.append(edge.start)
        coords.append(edge.end)
        if (abs(edge.start[0] - edge.end[0]) + abs(edge.start[1] - edge.end[1])) == 1:    
            prev_dir = (edge.end[0] - edge.start[0], edge.end[1] - edge.start[1])
        else:
            new_c = (edge.start[0] + prev_dir[0], edge.start[1] + prev_dir[1])
            coords.append(new_c)
            prev_dir = (edge.end[0] - new_c[0], edge.end[1] - new_c[1])

    return list(set(coords))

def get_path_coords_turns(p):
    coords = []
    prev_dir = (1, 0)
    for edge in p:
        if (abs(edge.start[0] - edge.end[0]) + abs(edge.start[1] - edge.end[1])) == 1:    
            prev_dir = (edge.end[0] - edge.start[0], edge.end[1] - edge.start[1])
        else:
            new_c = (edge.start[0] + prev_dir[0], edge.start[1] + prev_dir[1])
            coords.append(new_c)
            prev_dir = (edge.end[0] - new_c[0], edge.end[1] - new_c[1])

    return list(set(coords))


all_path_coords: set[tuple[int, int]] = set()
# original_path_coords: list[tuple[int, int]] = get_path_coords(path)

# for c in original_path_coords:
    # all_path_coords.add(c)

def same_coord_sets(c1, c2):
    return len(c1) == len(c2) and all(x in c1 for x in c2)

def same_paths(p1, p2):
    return same_coord_sets(get_path_coords(p1), get_path_coords(p2))

def same_paths_all(p1, list_p):
    for p2 in list_p:
        if same_paths(p1, p2):
            return True
    return False

def search_new_paths(path, pass_through: list, all_paths: list, maze_coordinates: dict[tuple[int, int], str], start, end, dist):
    path_coords = get_path_coords_turns(path)
    pass_throughs_copy = copy.deepcopy(pass_through)
    i = 0
    for c in path_coords:
        i += 1
        if c in pass_throughs_copy:
            continue
        maze_coords_copy = copy.deepcopy(maze_coordinates) # dict[tuple[int, int], str]
        maze_coords_copy[c] = '#'
        graph_copy = create_graph(maze_coords_copy)
        print(i)
        dijkstra_copy = DijkstraSP(graph_copy, start)
        if dijkstra_copy.has_path_to(end) == False or dijkstra_copy.dist_to_end(end) > dist:
            pass_throughs_copy.append(c)
            continue
        if dijkstra_copy.dist_to_end(end) == dist:
            path_new = dijkstra_copy.path_to(end)
            if same_paths_all(path_new, all_paths):
                continue

            all_paths.append(path_new)
            
            search_new_paths(path_new, pass_throughs_copy, all_paths, maze_coords_copy, start, end, dist)

all_paths = [path]
search_new_paths(path, [], all_paths, coordinates, start, end, dist)

count_coords = set()
for p in all_paths:
    # print_maze(maze, get_path_coords(p))
    # print()

    for c in get_path_coords(p):
        count_coords.add(c)

print(len(count_coords))





distance: 143580
1
2
3
4
5
6
7
8
9
2
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
71
72
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
89
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
1

In [13]:
print_maze(maze, get_path_coords_turns(path))

###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#....o...o.o#.#
###.#.#####.#.#
#...#o....#.#.#
#.#.#.###.#.#.#
#o.o.o#...#.#.#
#.###.#.#.#.#.#
#S..#.....#o.o#
###############


In [19]:
dijkstra = DijkstraSP(graph, start)

for k,v in dijkstra.edges_to.items():
    if len(v) > 1:
        print(k, v)

dijkstra.roadblock_locations()

(3, 8) [(2, 9) -[1002]-> (3, 8), (3, 10) -[2]-> (3, 8)]
(6, 7) [(5, 8) -[1002]-> (6, 7), (4, 7) -[2]-> (6, 7)]
(9, 11) [(9, 12) -[1]-> (9, 11), (9, 10) -[1]-> (9, 11)]


{(3, 8): [(2, 9), (3, 10)],
 (6, 7): [(5, 8), (4, 7)],
 (9, 11): [(9, 12), (9, 10)]}

In [26]:
dijkstra = DijkstraSP(graph, start)
total_distance = dijkstra.dist_to_end(end)
print("total distance:", total_distance)
crosses = dijkstra.roadblock_locations()
print(crosses)

all_path_coords = set()

def get_all_coords(start, crosses, coordinates, total):
    for k, cross in crosses.items():
        print(k)
        for coord in cross:
            if coord not in get_path_coords(dijkstra.path_to(end)):
                continue
            maze_copy = copy.deepcopy(coordinates)
            maze_copy[coord] = '#'
            g = create_graph(maze_copy)
            dijkstra_copy = DijkstraSP(g, start)
            
            if dijkstra_copy.dist_to_end(end) == total:
                p = dijkstra_copy.path_to(end)
                p_coords = get_path_coords(p)
                for c in p_coords:
                    all_path_coords.add(c)
    
    print(len(all_path_coords))

get_all_coords(start, crosses, coordinates, total_distance)
print_maze(maze, all_path_coords)

total distance: 7036
{(3, 8): [(2, 9), (3, 10)], (6, 7): [(5, 8), (4, 7)], (9, 11): [(9, 12), (9, 10)]}
(3, 8)
(6, 7)
(9, 11)
44
###############
#.......#....o#
#.#.###.#.###o#
#.....#.#...#o#
#.###.#####.#o#
#.#.#.......#o#
#.#.#####.###o#
#..ooooooooo#o#
###o#o#####o#o#
#ooo#o....#o#o#
#o#.#o###.#o#o#
#ooooo#...#o#o#
#o###.#.#.#o#o#
#o..#.....#ooo#
###############
